# Deep RL for Multi-Asset Portfolio Rebalancing

## Complete Training and Evaluation Pipeline

This notebook implements the full pipeline:
1. Data download and feature engineering
2. PPO training with validation-based model selection
3. Evaluation on held-out test set
4. Baseline comparisons (5 strategies)
5. Statistical tests (Diebold-Mariano, bootstrap CI)
6. Sensitivity analysis (cost and risk parameter sweeps)
7. Comprehensive visualizations

In [1]:
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Add parent directory to path
sys.path.append('..')

import numpy as np
import pandas as pd
import yaml
import json
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 100

print("Imports successful!")

Imports successful!


## 1. Load Configuration and Set Seeds

In [2]:
# Load config
with open('../config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Set seeds for reproducibility
seed = config['seed']
np.random.seed(seed)

try:
    import torch
    torch.manual_seed(seed)
    print(f"PyTorch seed set to {seed}")
except ImportError:
    print("PyTorch not installed, skipping torch seed")

print(f"\nConfiguration loaded:")
print(f"  Assets: {config['assets']}")
print(f"  Training period: {config['date']['train']}")
print(f"  Validation period: {config['date']['valid']}")
print(f"  Test period: {config['date']['test']}")
print(f"  Transaction cost: {config['trade']['cost_bps_per_turnover']} bps per turnover")
print(f"  Algorithm: {config['rl']['algo']}")

PyTorch seed set to 42

Configuration loaded:
  Assets: ['SPY', 'QQQ', 'IWM', 'EFA', 'EEM', 'TLT', 'HYG', 'GLD', 'DBC']
  Training period: ['2012-01-01', '2018-12-31']
  Validation period: ['2019-01-01', '2021-12-31']
  Test period: ['2022-01-01', '2025-10-31']
  Transaction cost: 25 bps per turnover
  Algorithm: PPO


## 2. Download Data

In [3]:
from data.download import fetch_ohlcv

tickers = config['assets'] + config['exogenous']
start_date = config['date']['train'][0]
end_date = config['date']['test'][1]

print(f"Downloading {len(tickers)} tickers from {start_date} to {end_date}...")
data = fetch_ohlcv(tickers, start_date, end_date)

print(f"\nDownloaded {len(data)} tickers successfully:")
for ticker, df in data.items():
    print(f"  {ticker}: {len(df)} days ({df.index.min().date()} to {df.index.max().date()})")

INFO:data.download:Loading SPY from cache: data\raw\SPY.parquet
INFO:data.download:Cached data for SPY doesn't cover range, re-downloading
INFO:data.download:Downloading SPY from 2012-01-01 to 2025-10-31
INFO:data.download:Cached SPY to data\raw\SPY.parquet
INFO:data.download:Loading QQQ from cache: data\raw\QQQ.parquet
INFO:data.download:Cached data for QQQ doesn't cover range, re-downloading
INFO:data.download:Downloading QQQ from 2012-01-01 to 2025-10-31
INFO:data.download:Cached QQQ to data\raw\QQQ.parquet
INFO:data.download:Loading IWM from cache: data\raw\IWM.parquet
INFO:data.download:Cached data for IWM doesn't cover range, re-downloading
INFO:data.download:Downloading IWM from 2012-01-01 to 2025-10-31
INFO:data.download:Cached IWM to data\raw\IWM.parquet
INFO:data.download:Loading EFA from cache: data\raw\EFA.parquet
INFO:data.download:Cached data for EFA doesn't cover range, re-downloading
INFO:data.download:Downloading EFA from 2012-01-01 to 2025-10-31
INFO:data.download:Cac


Downloaded 10 tickers successfully:
  SPY: 3478 days (2012-01-03 to 2025-10-30)
  QQQ: 3478 days (2012-01-03 to 2025-10-30)
  IWM: 3478 days (2012-01-03 to 2025-10-30)
  EFA: 3478 days (2012-01-03 to 2025-10-30)
  EEM: 3478 days (2012-01-03 to 2025-10-30)
  TLT: 3478 days (2012-01-03 to 2025-10-30)
  HYG: 3478 days (2012-01-03 to 2025-10-30)
  GLD: 3478 days (2012-01-03 to 2025-10-30)
  DBC: 3478 days (2012-01-03 to 2025-10-30)
  ^VIX: 3478 days (2012-01-03 to 2025-10-30)


## 3. Feature Engineering

In [4]:
from data.features import engineer_features

# Separate asset and exogenous data
asset_data = {k: v for k, v in data.items() if k in config['assets']}
exog_data = {k: v for k, v in data.items() if k in config['exogenous']}

print("Engineering features (causal, cross-sectionally normalized)...")
X, R = engineer_features(asset_data, exog_data, config)

print(f"\nFeature matrix shape: {X.shape}")
print(f"Return matrix shape: {R.shape}")
print(f"\nFeature columns ({len(X.columns)}):")
print(list(X.columns))

# Display sample
print("\nSample features (first date, first asset):")
print(X.head(1))

INFO:data.features:Engineering features for 9 assets with execution: next_open


Engineering features (causal, cross-sectionally normalized)...


INFO:data.features:Applying cross-sectional winsorization and z-scoring
INFO:data.features:Feature engineering complete: 31302 observations, 18 features
INFO:data.features:Return matrix shape: (3478, 9)



Feature matrix shape: (31302, 18)
Return matrix shape: (3478, 9)

Feature columns (18):
['ret_lag1', 'ret_lag2', 'ret_lag5', 'roll_mean_5', 'roll_mean_21', 'roll_mean_63', 'roll_std_5', 'roll_std_21', 'roll_std_63', 'rsi', 'macd', 'macd_signal', 'macd_hist', 'bb_pct', 'atr_norm', '^VIX_level', '^VIX_chg5', 'market_ret_lag1']

Sample features (first date, first asset):
                  ret_lag1  ret_lag2  ret_lag5  roll_mean_5  roll_mean_21  \
date       asset                                                            
2012-01-03 SPY         0.0       0.0       0.0          0.0           0.0   

                  roll_mean_63  roll_std_5  roll_std_21  roll_std_63  rsi  \
date       asset                                                            
2012-01-03 SPY             0.0         0.0          0.0          0.0  0.0   

                  macd  macd_signal  macd_hist  bb_pct  atr_norm  ^VIX_level  \
date       asset                                                               
2012

## 4. Create Train/Valid/Test Splits

In [5]:
from data.splits import make_splits, get_split_data

dates = X.index.get_level_values('date').unique()
splits = make_splits(dates, config, save_path='../data/splits.json')

# Extract split data
X_train, R_train, dates_train = get_split_data(X, R, splits['train'], config['assets'])
X_valid, R_valid, dates_valid = get_split_data(X, R, splits['valid'], config['assets'])
X_test, R_test, dates_test = get_split_data(X, R, splits['test'], config['assets'])

print(f"\nSplit shapes:")
print(f"  Train: X={X_train.shape}, R={R_train.shape}")
print(f"  Valid: X={X_valid.shape}, R={R_valid.shape}")
print(f"  Test:  X={X_test.shape}, R={R_test.shape}")

INFO:data.splits:Splits created and saved to ../data/splits.json
INFO:data.splits:Train: 1760 days (2012-01-03 to 2018-12-31)
INFO:data.splits:Valid: 757 days (2019-01-02 to 2021-12-31)
INFO:data.splits:Test: 961 days (2022-01-03 to 2025-10-30)



Split shapes:
  Train: X=(1760, 9, 18), R=(1760, 9)
  Valid: X=(757, 9, 18), R=(757, 9)
  Test:  X=(961, 9, 18), R=(961, 9)


## 5. Create Environments

In [6]:
from envs.portfolio_env import PortfolioEnv, rollout_policy

# Training environment
env_train = PortfolioEnv(X_train, R_train, dates_train, config)

print(f"Training environment created:")
print(f"  Observation space: {env_train.observation_space.shape}")
print(f"  Action space: {env_train.action_space.shape}")
print(f"  Episode length: {env_train.T} days")
print(f"  Transaction cost: {env_train.cost_rate*10000:.2f} bps per turnover")

INFO:envs.portfolio_env:PortfolioEnv initialized: T=1760, N=9, F=18, obs_dim=175


Training environment created:
  Observation space: (175,)
  Action space: (9,)
  Episode length: 1760 days
  Transaction cost: 25.00 bps per turnover


## 6. Train PPO Agent

In [7]:
from agents.ppo_trainer import train_ppo
import glob

# Validation evaluation function
def eval_fn(policy):
    """Rollout policy on validation set."""
    return rollout_policy(policy, X_valid, R_valid, dates_valid, config, deterministic=True)

# Check if a trained model already exists
log_dir = "../results/logs/ppo"
existing_models = glob.glob(f"{log_dir}/best_model_sharpe_*.zip")

if existing_models:
    # Use the best existing model (highest Sharpe in filename)
    best_model_path = max(existing_models, key=lambda p: float(p.split('_')[-1].replace('.zip', '')))
    print("\n" + "="*80)
    print("FOUND EXISTING TRAINED MODEL - SKIPPING TRAINING")
    print("="*80)
    print(f"Using existing model: {best_model_path}")
    print("\nTo retrain from scratch, delete files in: {log_dir}")
else:
    # Train PPO
    print("\n" + "="*80)
    print("TRAINING PPO AGENT")
    print("="*80)
    
    best_model_path = train_ppo(env_train, eval_fn, config, log_dir=log_dir)
    print(f"\nBest model saved to: {best_model_path}")


FOUND EXISTING TRAINED MODEL - SKIPPING TRAINING
Using existing model: ../results/logs/ppo\best_model_sharpe_1.3160.zip

To retrain from scratch, delete files in: {log_dir}


## 7. Evaluate RL Agent on Test Set

In [8]:
from stable_baselines3 import PPO

# Load best model
print("Loading best PPO model...")
print(f"Model path: {best_model_path}")
print(f"Model path type: {type(best_model_path)}")

# Ensure it's a string
if best_model_path is None:
    raise ValueError("No model was saved during training! Check if validation ran successfully.")

model_path_str = str(best_model_path)
print(f"Loading from: {model_path_str}")

ppo_model = PPO.load(model_path_str)

# Rollout on test set
print("Evaluating on test set...")
rl_results = rollout_policy(ppo_model, X_test, R_test, dates_test, config, deterministic=True)

print(f"\nRL Test Performance:")
print(f"  Days: {len(rl_results['daily_returns'])}")
print(f"  Mean daily return: {np.mean(rl_results['daily_returns'])*100:.4f}%")
print(f"  Daily vol: {np.std(rl_results['daily_returns'], ddof=1)*100:.4f}%")
print(f"  Avg turnover: {np.mean(rl_results['turnover'])*100:.2f}%")

Loading best PPO model...
Model path: ../results/logs/ppo\best_model_sharpe_1.3160.zip
Model path type: <class 'str'>
Loading from: ../results/logs/ppo\best_model_sharpe_1.3160.zip


INFO:envs.portfolio_env:PortfolioEnv initialized: T=961, N=9, F=18, obs_dim=175


Evaluating on test set...

RL Test Performance:
  Days: 960
  Mean daily return: 0.0168%
  Daily vol: 0.8134%
  Avg turnover: 4.21%


## 8. Run All Baselines on Test Set

In [9]:
from baselines import equal_weights, periodic_rebalance, risk_parity, momentum_tilt, mean_variance

baseline_results = {}

print("\n" + "="*80)
print("RUNNING BASELINES ON TEST SET")
print("="*80)

# Equal Weight Buy & Hold
print("\n1. Equal Weight Buy & Hold...")
baseline_results['EW_BuyHold'] = equal_weights.run_strategy(X_test, R_test, dates_test, config)

# Periodic Rebalance
print("2. Periodic Rebalance...")
baseline_results['Periodic_Rebal'] = periodic_rebalance.run_strategy(X_test, R_test, dates_test, config)

# Risk Parity
print("3. Risk Parity...")
baseline_results['Risk_Parity'] = risk_parity.run_strategy(X_test, R_test, dates_test, config)

# Momentum Tilt
print("4. Momentum Tilt...")
baseline_results['Momentum'] = momentum_tilt.run_strategy(X_test, R_test, dates_test, config)

# Mean-Variance
print("5. Mean-Variance...")
baseline_results['MeanVar'] = mean_variance.run_strategy(X_test, R_test, dates_test, config)

print("\nAll baselines complete!")


RUNNING BASELINES ON TEST SET

1. Equal Weight Buy & Hold...
2. Periodic Rebalance...
3. Risk Parity...
4. Momentum Tilt...
5. Mean-Variance...

All baselines complete!


## 9. Compute Performance Metrics

In [10]:
from metrics.evaluate import full_evaluation

# Collect all results
all_results = {
    'PPO_RL': rl_results,
    **baseline_results
}

# Compute metrics
metrics_summary = {}
for name, results in all_results.items():
    metrics_summary[name] = full_evaluation(results)

# Create summary DataFrame
df_metrics = pd.DataFrame(metrics_summary).T

# Reorder columns
cols_order = ['Sharpe', 'CAGR', 'Volatility', 'Sortino', 'Calmar', 
              'Max_Drawdown', 'Tail_Ratio', 'Annualized_Turnover', 'Cost_Drag_bps', 'HHI']
df_metrics = df_metrics[cols_order]

print("\n" + "="*80)
print("PERFORMANCE SUMMARY (TEST SET)")
print("="*80)
print(df_metrics.round(4))

# Save to CSV
df_metrics.to_csv('../results/test_performance_summary.csv')
print("\nSaved to results/test_performance_summary.csv")


PERFORMANCE SUMMARY (TEST SET)
                Sharpe    CAGR  Volatility  Sortino  Calmar  Max_Drawdown  \
PPO_RL          0.3284  0.0347      0.1291   0.4588  0.1192        0.2907   
EW_BuyHold      0.5492  0.0619      0.1230   0.7853  0.2907        0.2128   
Periodic_Rebal  0.5072  0.0573      0.1252   0.7274  0.2603        0.2199   
Risk_Parity     0.4846  0.0495      0.1128   0.6989  0.2292        0.2159   
Momentum        0.4004  0.0428      0.1239   0.5735  0.1850        0.2316   
MeanVar         0.2858  0.0259      0.1113   0.4001  0.1188        0.2185   

                Tail_Ratio  Annualized_Turnover  Cost_Drag_bps     HHI  
PPO_RL              0.9175              10.6000       264.9998  0.1666  
EW_BuyHold          0.9170               0.0000         0.0000  0.1148  
Periodic_Rebal      0.9539               0.1500         3.7490  0.1112  
Risk_Parity         0.9501               1.3725        34.3134  0.1325  
Momentum            0.9394               3.6048        90.1201 

In [11]:
# DIAGNOSTIC: Check baseline results
print("Checking baseline results...")
for name, results in baseline_results.items():
    daily_rets = results['daily_returns']
    print(f"\n{name}:")
    print(f"  Shape: {daily_rets.shape}")
    print(f"  Has NaN: {np.isnan(daily_rets).any()}")
    print(f"  Has Inf: {np.isinf(daily_rets).any()}")
    print(f"  Min: {np.min(daily_rets):.6f}")
    print(f"  Max: {np.max(daily_rets):.6f}")
    print(f"  Mean: {np.mean(daily_rets):.6f}")
    print(f"  Sample (first 5): {daily_rets[:5]}")

Checking baseline results...

EW_BuyHold:
  Shape: (960,)
  Has NaN: False
  Has Inf: False
  Min: -0.040646
  Max: 0.059543
  Mean: 0.000268
  Sample (first 5): [-1.57651203e-05 -1.44854063e-02  2.46168836e-04 -1.92183116e-03
 -1.27128657e-03]

Periodic_Rebal:
  Shape: (960,)
  Has NaN: False
  Has Inf: False
  Min: -0.039634
  Max: 0.058857
  Mean: 0.000252
  Sample (first 5): [-1.57651203e-05 -1.44854063e-02  2.46168836e-04 -1.92183116e-03
 -1.27128657e-03]

Risk_Parity:
  Shape: (960,)
  Has NaN: False
  Has Inf: False
  Min: -0.033092
  Max: 0.048289
  Mean: 0.000217
  Sample (first 5): [-1.57651203e-05 -1.45412379e-02  2.57411254e-04 -1.99207077e-03
 -1.26437481e-03]

Momentum:
  Shape: (960,)
  Has NaN: False
  Has Inf: False
  Min: -0.036925
  Max: 0.052547
  Mean: 0.000197
  Sample (first 5): [-1.57651203e-05 -1.45412379e-02  2.57411254e-04 -1.99207077e-03
 -1.26437481e-03]

MeanVar:
  Shape: (960,)
  Has NaN: False
  Has Inf: False
  Min: -0.034204
  Max: 0.042440
  Mean: 0.0

## 10. Statistical Tests

In [12]:
from metrics.tests import diebold_mariano, sharpe_block_bootstrap

print("\n" + "="*80)
print("STATISTICAL TESTS")
print("="*80)

# 1. Diebold-Mariano: RL vs best baseline
best_baseline = df_metrics.drop('PPO_RL').sort_values('Sharpe', ascending=False).index[0]
print(f"\n1. Diebold-Mariano Test: PPO_RL vs {best_baseline}")

dm_result = diebold_mariano(
    rl_results['daily_returns'],
    all_results[best_baseline]['daily_returns'],
    loss='neg_return'
)

for key, value in dm_result.items():
    print(f"  {key}: {value}")

# 2. Block Bootstrap CI for RL Sharpe
print("\n2. Block Bootstrap 95% CI for PPO_RL Sharpe")
boot_result = sharpe_block_bootstrap(rl_results['daily_returns'], block=20, reps=5000)

for key, value in boot_result.items():
    if isinstance(value, (int, float)):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")


STATISTICAL TESTS

1. Diebold-Mariano Test: PPO_RL vs EW_BuyHold
  DM_statistic: 1.1828749487168468
  p_value: 0.236858711280155
  mean_loss_diff: 9.988915137004086e-05
  interpretation: X better than Y

2. Block Bootstrap 95% CI for PPO_RL Sharpe
  observed_sharpe: 0.3284
  ci_lower_95: -0.6454
  ci_upper_95: 1.4109
  bootstrap_mean: 0.3823
  bootstrap_std: 0.5282


## 11. Visualizations

In [13]:
from plots.equity import plot_multiple_equity_curves, plot_equity_curve
from plots.rolling import plot_rolling_sharpe, plot_drawdown
from plots.weights import plot_weights_heatmap, plot_weights_area, plot_weight_statistics
from plots.sensitivity import plot_turnover_vs_sharpe

print("Generating plots...")

# 1. Multiple equity curves
plot_multiple_equity_curves(
    all_results,
    title="Strategy Comparison: Equity Curves (Test Set)",
    savepath="../figures/equity_curves_comparison.png"
)

# 2. RL equity curve with drawdown
plot_equity_curve(
    rl_results['daily_returns'],
    title="PPO RL Agent: Equity Curve",
    savepath="../figures/rl_equity_curve.png",
    dates=dates_test
)

# 3. Rolling Sharpe
plot_rolling_sharpe(
    rl_results['daily_returns'],
    window=63,
    savepath="../figures/rl_rolling_sharpe.png",
    dates=dates_test
)

# 4. Drawdown
plot_drawdown(
    rl_results['daily_returns'],
    savepath="../figures/rl_drawdown.png",
    dates=dates_test
)

# 5. Weights heatmap
plot_weights_heatmap(
    rl_results['weights'],
    config['assets'],
    savepath="../figures/rl_weights_heatmap.png"
)

# 6. Weights area chart
plot_weights_area(
    rl_results['weights'],
    config['assets'],
    savepath="../figures/rl_weights_area.png"
)

# 7. Weight statistics
plot_weight_statistics(
    rl_results['weights'],
    config['assets'],
    savepath="../figures/rl_weight_stats.png"
)

# 8. Turnover vs Sharpe
plot_turnover_vs_sharpe(
    all_results,
    savepath="../figures/turnover_vs_sharpe.png"
)

print("\nAll plots saved to figures/ directory!")

Generating plots...
Saved comparison plot to ../figures/equity_curves_comparison.png
Saved equity plot to ../figures/rl_equity_curve.png
Saved rolling Sharpe plot to ../figures/rl_rolling_sharpe.png
Saved drawdown plot to ../figures/rl_drawdown.png
Saved weights heatmap to ../figures/rl_weights_heatmap.png
Saved weights area plot to ../figures/rl_weights_area.png
Saved weight statistics plot to ../figures/rl_weight_stats.png
Saved turnover vs sharpe plot to ../figures/turnover_vs_sharpe.png

All plots saved to figures/ directory!


## 12. Sensitivity Analysis: Transaction Cost Sweep

In [14]:
from plots.sensitivity import plot_cost_sweep

print("\n" + "="*80)
print("SENSITIVITY ANALYSIS: TRANSACTION COST SWEEP")
print("="*80)

cost_sweep_values = [0, 5, 10, 20, 30, 50]  # bps per turnover
cost_sweep_results = []

for cost_bps in cost_sweep_values:
    print(f"\nTesting cost = {cost_bps} bps...")
    
    # Update config
    config_sweep = config.copy()
    config_sweep['trade']['cost_bps_per_turnover'] = cost_bps
    
    # Re-run strategies (using cached weights, just recompute returns with new cost)
    for strategy_name, cached_results in all_results.items():
        weights = cached_results['weights']
        turnover = cached_results['turnover']
        
        # Recompute with new cost
        cost_rate = cost_bps / 10000.0
        costs = turnover * cost_rate
        
        # Recompute net returns
        daily_returns = []
        for t in range(len(turnover)):
            gross_ret = np.dot(weights[t], R_test[t])
            net_ret = gross_ret - costs[t]
            daily_returns.append(net_ret)
        
        daily_returns = np.array(daily_returns)
        sharpe = (np.mean(daily_returns) / (np.std(daily_returns, ddof=1) + 1e-10)) * np.sqrt(252)
        
        cost_sweep_results.append({
            'cost_bps': cost_bps,
            'strategy': strategy_name,
            'sharpe': sharpe
        })

df_cost_sweep = pd.DataFrame(cost_sweep_results)
print("\nCost Sweep Results:")
print(df_cost_sweep.pivot(index='cost_bps', columns='strategy', values='sharpe').round(3))

# Plot
plot_cost_sweep(df_cost_sweep, savepath="../figures/cost_sensitivity.png")
print("\nCost sensitivity plot saved!")


SENSITIVITY ANALYSIS: TRANSACTION COST SWEEP

Testing cost = 0 bps...

Testing cost = 5 bps...

Testing cost = 10 bps...

Testing cost = 20 bps...

Testing cost = 30 bps...

Testing cost = 50 bps...

Cost Sweep Results:
strategy  EW_BuyHold  MeanVar  Momentum  PPO_RL  Periodic_Rebal  Risk_Parity
cost_bps                                                                    
0              0.549    0.312     0.473   0.534           0.510        0.515
5              0.549    0.307     0.459   0.493           0.510        0.509
10             0.549    0.301     0.444   0.452           0.509        0.503
20             0.549    0.291     0.415   0.369           0.508        0.491
30             0.549    0.281     0.386   0.287           0.507        0.478
50             0.549    0.260     0.328   0.123           0.504        0.454
Saved cost sweep plot to ../figures/cost_sensitivity.png

Cost sensitivity plot saved!


## 13. Export Results

In [15]:
print("\n" + "="*80)
print("EXPORTING RESULTS")
print("="*80)

# Get the actual number of returns (should be 960)
n_returns = len(rl_results['daily_returns'])

# 1. Daily returns - slice dates to match returns length
df_returns = pd.DataFrame({
    name: results['daily_returns'] 
    for name, results in all_results.items()
}, index=dates_test[:n_returns])
df_returns.to_csv('../results/test_daily_returns.csv')
print("\n1. Saved: results/test_daily_returns.csv")

# 2. Weights (RL only) - slice dates to match weights length
df_weights = pd.DataFrame(
    rl_results['weights'],
    columns=config['assets'],
    index=dates_test[:n_returns]
)
df_weights.to_csv('../results/test_weights_rl.csv')
print("2. Saved: results/test_weights_rl.csv")

# 3. Artifacts metadata
artifacts = {
    'config': config,
    'metrics_summary': df_metrics.to_dict(),
    'dm_test': dm_result,
    'bootstrap_ci': boot_result,
    'library_versions': {
        'numpy': np.__version__,
        'pandas': pd.__version__,
    }
}

try:
    import torch
    artifacts['library_versions']['torch'] = torch.__version__
except:
    pass

with open('../results/artifacts.json', 'w') as f:
    json.dump(artifacts, f, indent=2, default=str)
print("3. Saved: results/artifacts.json")

print("\n" + "="*80)
print("ALL DONE! 🎉")
print("="*80)
print("\nResults saved to:")
print("  - results/test_daily_returns.csv")
print("  - results/test_weights_rl.csv")
print("  - results/test_performance_summary.csv")
print("  - results/artifacts.json")
print("\nFigures saved to:")
print("  - figures/equity_curves_comparison.png")
print("  - figures/rl_*.png (various plots)")
print("  - figures/cost_sensitivity.png")
print("\nBest model saved to:")
print(f"  - {best_model_path}")


EXPORTING RESULTS

1. Saved: results/test_daily_returns.csv
2. Saved: results/test_weights_rl.csv
3. Saved: results/artifacts.json

ALL DONE! 🎉

Results saved to:
  - results/test_daily_returns.csv
  - results/test_weights_rl.csv
  - results/test_performance_summary.csv
  - results/artifacts.json

Figures saved to:
  - figures/equity_curves_comparison.png
  - figures/rl_*.png (various plots)
  - figures/cost_sensitivity.png

Best model saved to:
  - ../results/logs/ppo\best_model_sharpe_1.3160.zip
